# KDD Cup 1999 Data(標準化、次元圧縮、SVM)

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from sklearn import datasets
from numpy import logspace
import sklearn.preprocessing as sp
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
import sklearn
from sklearn.decomposition import PCA
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
% matplotlib inline

/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.



|ファイル名|ファイル内容|
|---|---|
|kddcup.data|フルデータ|
|kddcup.data_10_percent|フルデータの10%を抽出した学習用データ|
|corrected|正常・攻撃のラベル付けがなされた評価用データ|
|kddcup.testdata.unlabeled|正常・攻撃のラベル付けがなされていないデータ|
|kddcup.testdata.unlabeled_10_percent|正常・攻撃のラベル付けがなされていないデータの10%サブセット|
|kddcup.newtestdata_10_percent_unlabeled|正常・攻撃のラベル付けがなされていないデータの10%サブセット|

In [3]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [4]:
kdd_data_10percent = joblib.load('dump/kddcup.data/kdd_data_10percent.00.pkl')


In [5]:
kdd_data_10percent.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,24,9,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,1,24,9,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,1,24,9,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,1,24,9,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,1,24,9,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [6]:
train_features = kdd_data_10percent.drop(['label','service'],axis=1)

In [7]:
train_labels = kdd_data_10percent['label'].copy()

In [8]:
train_labels.head()

0    normal.
1    normal.
2    normal.
3    normal.
4    normal.
Name: label, dtype: object

In [9]:
#Pipeline(steps=[
#('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), 
#('pca', PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,svd_solver='auto', tol=0.0, whiten=False)), 
#('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=None, degree=3, gamma=0.005623413251903491,kernel='rbf', max_iter=-1, probability=False, random_state=None,shrinking=True, tol=0.001, verbose=False))])    

pipeline = Pipeline([
        ('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
    ('pca', PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,svd_solver='auto', tol=0.0, whiten=False)),
        ('svm', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=None, degree=3, gamma=0.005623413251903491,kernel='rbf', max_iter=-1, probability=False, random_state=None,shrinking=True, tol=0.001, verbose=False))])

In [10]:
t1=time.perf_counter()
pipeline.fit(train_features, train_labels)
t2=time.perf_counter()

In [11]:
print(t2-t1,"秒")

27.393579296011012 秒


In [12]:
t1=time.perf_counter()
pred = pipeline.predict(train_features)
t2=time.perf_counter()


In [13]:
print(t2-t1,"秒")

85.4145835340023 秒


In [14]:
print(classification_report(train_labels, pred))
print(confusion_matrix(train_labels, pred))

                  precision    recall  f1-score   support

           back.       0.98      0.99      0.99      2203
buffer_overflow.       1.00      0.60      0.75        30
      ftp_write.       1.00      0.50      0.67         8
   guess_passwd.       1.00      1.00      1.00        53
           imap.       0.50      0.08      0.14        12
        ipsweep.       0.98      0.98      0.98      1247
           land.       0.95      1.00      0.98        21
     loadmodule.       1.00      0.33      0.50         9
       multihop.       0.80      0.57      0.67         7
        neptune.       1.00      1.00      1.00    107201
           nmap.       0.97      0.87      0.92       231
         normal.       0.99      1.00      0.99     97278
           perl.       1.00      1.00      1.00         3
            phf.       1.00      1.00      1.00         4
            pod.       0.99      0.65      0.79       264
      portsweep.       0.99      0.98      0.98      1040
        rootk

In [15]:
#joblib.dump(pipeline, 'dump/20171111/clf2.pkl') 

['dump/20171111/clf2.pkl']